In [ ]:
import tensorflow as tf
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.8.0


Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Select Model

In [ ]:
model_location = "/content/drive/MyDrive/CV_Research/StairNet_Results/Project1_Final_Model.h5"

Create  Generator to generate data for Quantization (To do later for use as calibration data)

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# project_id = 'stairnet'
# !gcloud config set project {project_id}

In [ ]:
# # Test to see if dataset location is correct
# ! gsutil ls -al gs://stairnet_bucket

In [ ]:
# GCS_DS_PATH = 'gs://stairnet_bucket'

In [ ]:
# GCS_PATH_SELECT = { # available image sizes
#     256: GCS_DS_PATH + '/Split_StairNet_TFRecord',
# }
# GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]
# print(GCS_PATH)


# TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
# VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
# TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') # predictions on this dataset should be submitted for the competition

# CLASSES = ['IS', 'IS-LG', 'LG', 'LG-IS']  

In [ ]:
# def decode_image(image_data):
#     image = tf.io.decode_raw(image_data, tf.uint8)
#     # image = tf.image.decode_jpeg(image_data, channels=3)
#     # image = image_data
#     image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
#     image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
#     image = tf.image.random_crop(value=image, size=(224, 224, 3))
#     return image

# def read_labeled_tfrecord(example):
#     LABELED_TFREC_FORMAT = {
#         "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
#         "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
#     }
#     example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
#     image = decode_image(example['image'])
#     label = tf.cast(example['label'], tf.int32)
#     return image, label # returns a dataset of (image, label) pairs

# def read_unlabeled_tfrecord(example):
#     UNLABELED_TFREC_FORMAT = {
#         "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
#         "label": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
#     }
#     example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
#     image = decode_image(example['image'])
#     idnum = example['label']
#     return image, idnum # returns a dataset of image(s)

# def load_dataset(filenames, labeled=True, ordered=False):
#     # Read from TFRecords. For optimal performance, reading from multiple files at once and
#     # disregarding data order. Order does not matter since we will be shuffling the data anyway.

#     ignore_order = tf.data.Options()
#     if not ordered:
#         ignore_order.experimental_deterministic = False # disable order, increase speed

#     dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
#     dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
#     dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
#     # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
#     return dataset

# def data_augment(image, label):
#     # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
#     # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
#     # of the TPU while the TPU itself is computing gradients.
#     image = tf.image.random_flip_left_right(image)
#     #image = tf.image.random_saturation(image, 0, 2)
#     return image, label   

# def get_training_dataset():
#     dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
#     dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
#     dataset = dataset.repeat() # the training dataset must repeat for several epochs
#     # for hyperparameter testing using random seed to shuffle the data the same to elimite this variable from results
#     dataset = dataset.shuffle(count_data_items(TRAINING_FILENAMES)/10, seed=SEED_NUMBER, reshuffle_each_iteration=None)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def get_validation_dataset(ordered=False):
#     dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     # for hyperparameter testing using random seed to shuffle the data the same to elimite this variable from results
#     dataset = dataset.shuffle(count_data_items(VALIDATION_FILENAMES)/10, seed=SEED_NUMBER, reshuffle_each_iteration=None)
#     dataset = dataset.cache()
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def get_test_dataset(ordered=False):
#     dataset = load_dataset(TEST_FILENAMES, labeled=True, ordered=ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     # for hyperparameter testing using random seed to shuffle the data the same to elimite this variable from results
#     dataset = dataset.shuffle(count_data_items(TEST_FILENAMES)/10, seed=SEED_NUMBER, reshuffle_each_iteration=None)
#     dataset = dataset.cache()
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset 

# def count_data_items(filenames):
#     n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
#     return np.sum(n)

# NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
# NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
# NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
# VALIDATION_STEPS = -(-NUM_VALIDATION_IMAGES // BATCH_SIZE) # The "-(-//)" trick rounds up instead of down :-)
# TEST_STEPS = -(-NUM_TEST_IMAGES // BATCH_SIZE)             # The "-(-//)" trick rounds up instead of down :-)
# print('Dataset: {} training images, {} validation images, {} test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
# def generator():
#   data = get_validation_dataset()
#   data = data.unbatch().batch(20)
#   for _ in range(num_calibration_steps):  
#     image, = data.take(1)
#     yield [image]

Convert TF model to TF Lite

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model_file( '/content/drive/MyDrive/CV_Research/StairNet_Results/Project1_Final_Model.h5' ) # Your model's name
# model = converter.convert()
# file = open( 'model.tflite' , 'wb' ) 
# file.write( model )

# WHOLE MODEL
tflite_model = tf.keras.models.load_model('/content/drive/MyDrive/CV_Research/StairNet_Results/StairNet_Model_ICORR_Final.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_save = converter.convert()
open("generated.tflite", "wb").write(tflite_save)

INFO:tensorflow:Assets written to: /tmp/tmpe2wplgnk/assets


INFO:tensorflow:Assets written to: /tmp/tmpe2wplgnk/assets


4485984